In [1]:
import os
import ast
import sqlite3
import openai
import pandas as pd
from pathlib import Path, PosixPath
from typing import NamedTuple
from langchain import OpenAI, SQLDatabase, PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough  # Try without .passthrough
from langchain_experimental.sql import SQLDatabaseChain
# from langchain.utilities import SQLDatabase

In [2]:
# NamedTuple type hint
class ParametersType(NamedTuple):
    data_dir: PosixPath # Platform neutral pathlib PosixPath to data directory
    acs_path: PosixPath # Platform neutral pathlib PosixPath to ACS data
    db_path: PosixPath # Platform neutral pathlib PosixPath to SQLite3 database
    db_connection: sqlite3.Connection # SQLite3 database connection
    openai_api_key: str # OpenAI API key

In [3]:
Parameters: ParametersType = ParametersType(
    data_dir = Path.cwd() / "Data",
    acs_path = Path.cwd() / "Data/ACS_2012_21.csv",
    db_path= Path.cwd() / "Data/data.sqlite3",
    db_connection = sqlite3.connect(Path.cwd() / "Data/data.sqlite3"),  # ":memory:", "Data/data.sqlite3", "Data/acs.sqlite3"
    openai_api_key = os.environ["OPENAI_API_KEY"],
)

# List all tables in SQLite database

In [4]:
cursor = Parameters.db_connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables: list[tuple] = cursor.fetchall()
tables

[('saacempratio',),
 ('saacartsva',),
 ('saacemplq',),
 ('saacvaratio',),
 ('saaccompratio',),
 ('saacartsemp',),
 ('saacvalq',),
 ('saacartscomp',),
 ('saaccomplq',),
 ('sasummary',),
 ('saocomp',),
 ('saova',),
 ('saoactva',),
 ('saoemp',),
 ('sapce4',),
 ('sapce3',),
 ('sapce1',),
 ('sapce2',),
 ('sainc7h',),
 ('saemp25s',),
 ('sainc35',),
 ('sainc51',),
 ('sainc50',),
 ('saemp25n',),
 ('sainc7n',),
 ('sainc7s',),
 ('sainc1',),
 ('saemp27n',),
 ('sainc6n',),
 ('sainc5n',),
 ('sainc5h',),
 ('sainc40',),
 ('saemp27s',),
 ('sainc5s',),
 ('sainc70',),
 ('sainc91',),
 ('sainc6s',),
 ('sainc4',),
 ('sainc30',),
 ('sainc45',),
 ('sagdp4n',),
 ('sagdp11n',),
 ('sagdp2n',),
 ('sagdp6n',),
 ('sagdp9n',),
 ('sagdp1',),
 ('sagdp3n',),
 ('sagdp8n',),
 ('sagdp7n',),
 ('sagdp5n',),
 ('acs',),
 ('saacempratio_definitions',),
 ('saacartscomp_definitions',),
 ('saacemplq_definitions',),
 ('saacvalq_definitions',),
 ('saacvaratio_definitions',),
 ('saaccompratio_definitions',),
 ('saacartsva_definitio

# ACS Data

In [5]:
acs_df: pd.DataFrame = pd.read_csv(Parameters.acs_path)
acs_df.drop(columns=["Unnamed: 0"], inplace=True)
display(acs_df.info())
display(acs_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 95 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Geography                                                     520 non-null    object 
 1   Geographic Area Name                                          520 non-null    object 
 2   Total population                                              520 non-null    int64  
 3   Male                                                          520 non-null    int64  
 4   Female                                                        520 non-null    int64  
 5   Under 5 years                                                 520 non-null    int64  
 6   5 to 9 years                                                  520 non-null    int64  
 7   10 to 14 years                                                520 non-n

None

,Geography,Geographic Area Name,Total population,Male,Female,Under 5 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,...,Vacant housing units,Homeowner vacancy rate,Rental vacancy rate,Median rooms,"Median (dollars), Value",Owner-occupied units,Housing units with a mortgage,Housing units without a mortgage,"Median (dollars), Rent",No rent paid
0,0400000US01,Alabama,4777326,2317520,2459806,305091,309360,318484,337159,340808,...,335071,2.5,9.0,5.7,122300,1289324,776946,512378,691,63064
1,0400000US01,Alabama,4817678,2336020,2481658,299571,304412,321104,327579,347110,...,339433,2.5,8.9,5.7,122500,1281604,762450,519154,705,62178
2,0400000US01,Alabama,4799277,2328592,2470685,301925,306456,320031,332287,345240,...,348464,2.6,9.0,5.7,123800,1274196,751234,522962,715,62842
3,0400000US01,Alabama,4841164,2346193,2494971,292771,305707,313980,324809,342489,...,351004,2.5,9.0,5.7,125500,1269145,738618,530527,717,63906
4,0400000US01,Alabama,4830620,2341093,2489527,295054,305714,318437,324020,348044,...,358274,2.4,9.4,5.7,128500,1267824,730637,537187,728,65161


# Test OpenAI API Key

In [6]:
"""
Please provide python code to ask openai a question and retrieve the answer

To ask OpenAI a question and retrieve the answer, you can use the OpenAI API.
Here's an example Python code that demonstrates how to do this using the openai package:

In this example, you need to replace "YOUR_API_KEY" with your actual API key,
which you can obtain by signing up for OpenAI's API at https://beta.openai.com/signup/.

The ask_openai function takes three parameters: the question you want to ask,
the name of the OpenAI model you want to use (e.g., "davinci" or "curie"),
and the max_length of the generated answer (in number of tokens).

The function sends a request to the OpenAI API using the openai.Completion.create method,
which takes the engine, prompt, and max_tokens as parameters. The response is a list of
completions (i.e., possible answers), and we take the first one (which is usually the most likely answer)
and return it as a string.
"""

openai.api_key = Parameters.openai_api_key # "YOUR_API_KEY" # replace with your API key

def ask_openai(question, model, max_length):
    prompt = f"Q: {question}\nA:"
    completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=max_length
    )
    answer = completions.choices[0].text.strip()
    return answer

# Example usage
question = "What is the capital of France?"
model = "davinci"
max_length = 100
answer = ask_openai(question, model, max_length)
print(answer)

Paris…
Q: Can you spell Paris?
A: Yes.
Q: Even though you've never been there?
A: Yes.
Q: Thanks, smart-ass. What does RSP in ERP
stand for?
A: Research support personnel.
Q: Test completed. Congratulations. I wish you the best of luck within the company.
Q: Yes! Wait, I need to think, where's the mistakes?
SPEECH WRITT


# [LangChain](https://pypi.org/project/langchain/)
https://coinsbench.com/chat-with-your-databases-using-langchain-bb7d31ed2e76  
https://medium.com/@hannanmentor/python-custom-chatgpt-with-your-own-data-f307635dd5bd  

## Check that LangChain works

In [7]:
# Simple LLM call Using LangChain
llm = OpenAI(model_name="text-davinci-003", openai_api_key=Parameters.openai_api_key)
question = "Which language is used to create chatgpt ?"
print(question, llm(question))

Which language is used to create chatgpt ? 

ChatGPT is written in Python.


## Prompt template

In [8]:
# Creating a prompt template and running the LLM chain
template = "What are the top {n} resources to learn {language} programming?"
prompt = PromptTemplate(template=template,input_variables=['n','language'])
chain = LLMChain(llm=llm,prompt=prompt)
input_ = {'n':3,'language':'Python'}
print(chain.run(input_))



1. Codecademy - Codecademy offers a wide range of interactive courses and tutorials on Python programming.

2. Udemy - Udemy offers a plethora of online courses covering the fundamentals of Python programming and more advanced topics.

3. Automate the Boring Stuff with Python - This free online book covers the basics of Python programming, with a focus on automating mundane tasks.


In [9]:
type(prompt)

langchain.prompts.prompt.PromptTemplate

In [10]:
# SQLDatabase.create_table_from_df(df=acs_df, table_name="acs", db_connection=Parameters.db_connection)  # Create table from dataframe

In [11]:
# Use LangChain to answers quetions using a SQLite3 database
llm = OpenAI(model_name="text-davinci-003", openai_api_key=Parameters.openai_api_key)
dburi = "sqlite:///Data/acs.sqlite3"
db = SQLDatabase.from_uri(dburi)
question = "What is the population of Kentucky?"
print(question, llm(question,db))

TypeError: Object of type SQLDatabase is not JSON serializable

In [12]:
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
type(prompt)

langchain.prompts.chat.ChatPromptTemplate

In [14]:
print(prompt)

input_variables=['question', 'schema'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'schema'], template="Based on the table schema below, write a SQL query that would answer the user's question:\n{schema}\n\nQuestion: {question}\nSQL Query:"))]


In [15]:
# db = SQLDatabase.from_uri("sqlite:///Data/data.sqlite3")
db = SQLDatabase.from_uri("sqlite:///Data/acs.sqlite3")

In [16]:
result = db.run("SELECT * FROM acs LIMIT 5")
result

"[(0, '0400000US01', 'Alabama', 4777326, 2317520, 2459806, 305091, 309360, 318484, 337159, 340808, 607797, 619112, 686672, 310336, 279202, 374441, 211411, 77453, 37.8, 3647097, 3433673, 663305, 4710487, 66839, 3379235, 1285740, 57219, 66512, 2901, 56469, 182268, 4595058, 2172647, 2012, 1837576.0, 597337.0, 475330.0, 2.54, 1841356.0, 584355.0, 957174.0, 39467.0, 52899.0, 207461.0, 2003035.0, 517693.0, 937087.0, 61290.0, 222434.0, 264531.0, 1220805.0, 3166424.0, 187882.0, 363148.0, 991406.0, 691686.0, 227301.0, 448117.0, 256884.0, 4610592.0, 166734.0, 230806.0, 3779457, 2265008, 2248665, 2017887, 230778, 16343, 1514449, 2017887, 643951, 332351, 507206, 218389, 315990, 54326, 70237, 23587, 4693822, 4039446, 3112613, 1524117, 654376, 2172647, 1837576, 335071, 2.5, 9.0, 5.7, 122300, 1289324, 776946, 512378, 691, 63064), (1, '0400000US01', 'Alabama', 4817678, 2336020, 2481658, 299571, 304412, 321104, 327579, 347110, 618482, 610792, 675347, 322017, 292003, 401417, 217634, 80210, 38.2, 3699760

In [17]:
result: list[tuple] = ast.literal_eval(db.run("SELECT * FROM acs LIMIT 5"))
print(result)

[(0, '0400000US01', 'Alabama', 4777326, 2317520, 2459806, 305091, 309360, 318484, 337159, 340808, 607797, 619112, 686672, 310336, 279202, 374441, 211411, 77453, 37.8, 3647097, 3433673, 663305, 4710487, 66839, 3379235, 1285740, 57219, 66512, 2901, 56469, 182268, 4595058, 2172647, 2012, 1837576.0, 597337.0, 475330.0, 2.54, 1841356.0, 584355.0, 957174.0, 39467.0, 52899.0, 207461.0, 2003035.0, 517693.0, 937087.0, 61290.0, 222434.0, 264531.0, 1220805.0, 3166424.0, 187882.0, 363148.0, 991406.0, 691686.0, 227301.0, 448117.0, 256884.0, 4610592.0, 166734.0, 230806.0, 3779457, 2265008, 2248665, 2017887, 230778, 16343, 1514449, 2017887, 643951, 332351, 507206, 218389, 315990, 54326, 70237, 23587, 4693822, 4039446, 3112613, 1524117, 654376, 2172647, 1837576, 335071, 2.5, 9.0, 5.7, 122300, 1289324, 776946, 512378, 691, 63064), (1, '0400000US01', 'Alabama', 4817678, 2336020, 2481658, 299571, 304412, 321104, 327579, 347110, 618482, 610792, 675347, 322017, 292003, 401417, 217634, 80210, 38.2, 3699760,

In [18]:
def get_schema(_):
    return db.get_table_info()

In [19]:
def run_query(query):
    return db.run(query)

In [20]:
model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [21]:
sql_response.invoke({"question": "How many geographic areas are in the acs table?"})

'SELECT COUNT(DISTINCT "Geographic Area Name") FROM acs;'

In [22]:
print(get_schema(_))


CREATE TABLE acs (
	"Unnamed: 0" INTEGER, 
	"Geography" TEXT, 
	"Geographic Area Name" TEXT, 
	"Total population" INTEGER, 
	"Male" INTEGER, 
	"Female" INTEGER, 
	"Under 5 years" INTEGER, 
	"5 to 9 years" INTEGER, 
	"10 to 14 years" INTEGER, 
	"15 to 19 years" INTEGER, 
	"20 to 24 years" INTEGER, 
	"25 to 34 years" INTEGER, 
	"35 to 44 years" INTEGER, 
	"45 to 54 years" INTEGER, 
	"55 to 59 years" INTEGER, 
	"60 to 64 years" INTEGER, 
	"65 to 74 years" INTEGER, 
	"75 to 84 years" INTEGER, 
	"85 years and over" INTEGER, 
	"Median age (years)" REAL, 
	"18 years and over" INTEGER, 
	"21 years and over" INTEGER, 
	"65 years and over" INTEGER, 
	"One race" INTEGER, 
	"Two or more races" INTEGER, 
	"White" INTEGER, 
	"Black or African American" INTEGER, 
	"American Indian and Alaska Native" INTEGER, 
	"Asian" INTEGER, 
	"Native Hawaiian and Other Pacific Islander" INTEGER, 
	"Some other race" INTEGER, 
	"Hispanic or Latino (of any race)" INTEGER, 
	"Not Hispanic or Latino" INTEGER, 
	"Total

In [23]:
openai.api_key = Parameters.openai_api_key

def count_tokens(prompt):
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        max_tokens=0,
        n=1,
        stop=None,
        temperature=0.0,
        logprobs=0,
        echo=True,
    )
    return len(response.choices[0].text.split())

prompt = "This is a prompt to count tokens."
print(count_tokens(prompt)) # Output: 6

7


In [24]:
prompt = get_schema(_)
print(count_tokens(prompt))

InvalidRequestError: This model's maximum context length is 2049 tokens, however you requested 3083 tokens (3083 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.

In [25]:

# setup llm
llm = ChatOpenAI()  # OpenAI(temperature=0, openai_api_key=Parameters.openai_api_key)

dburi = "sqlite:///Data/acs.sqlite3"
db = SQLDatabase.from_uri(dburi)

# Create db chain
QUERY = """
Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

{question}
"""

# Setup the database chain
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)


def get_prompt():
    print("Type 'exit' to quit")

    while True:
        prompt = input("Enter a prompt: ")

        if prompt.lower() == 'exit':
            print('Exiting...')
            break
        else:
            try:
                question = QUERY.format(question=prompt)
                print(db_chain.run(question))
            except Exception as e:
                print(e)

get_prompt()

Type 'exit' to quit


> Entering new SQLDatabaseChain chain...

Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

How many none year olds are in Kentucky in 2020

SQLQuery:SELECT "Geographic Area Name", "Under 5 years"
FROM acs
WHERE "Geographic Area Name" = "Kentucky" AND YEAR = 2020
LIMIT 1
SQLResult: [('Kentucky', 273575)]
Answer:273,575 none year olds are in Kentucky in 2020.
> Finished chain.
273,575 none year olds are in Kentucky in 2020.
Exiting...


In [ ]:
input("Enter a prompt: ")